In [1]:
import numpy as np
import cv2
import os
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg
from IPython.display import display, clear_output
import ipywidgets
from jetracer.nvidia_racecar import NvidiaRacecar
from marvelmind import MarvelmindHedge
import urllib3, json
import threading
import time

camera = CSICamera(width=1280, height=720)
camera.running = True

In [2]:
left_s, right_s = (False, False)
updated = False
last_fit = False

def whiteMask(image):
    lower = np.array([155, 150, 110])
    upper = np.array([255, 255, 255])
    mask = cv2.inRange(image, lower, upper)
    return mask

def yellowMask(image):
    lower = np.array([0, 150, 0])
    upper = np.array([150, 255, 255])
    mask = cv2.inRange(image, lower, upper)
    return mask

def parse_steering(x, mid, l_offset, r_offset, bools):
    """
        x -> list from sliding window
        mid -> line middle location
        l/r_offset -> offset from left/right line to center
        bools -> left, right, mid        
    """
    l, r, m = bools
    steer = 0
    
    if l or r:
        if l:
            lane_mid = np.min(x)+l_offset
        elif r:
            lane_mid = np.max(x)-r_offset
        steer = ((mid-lane_mid)/(mid))
    elif m:
        if (np.mean(x)<x[-1]):
            steer = -0.65
        else:
            steer = 0.65
        
    return steer

def line_check(img, n = 130):
    """
        img -> image mask
        n -> minimum line height
    """
    thr = 2550 #threshold
    height = 0
    width = 0
    y = []
    hist_h = np.sum(img, axis=1)
    hist_w = np.sum(img, axis=0)
    
    for i in range(len(hist_h)):
        if hist_h[i]>thr:
            height+=1
            y.append(i)

    l = False if height<n else True
    
    if l and len(y)>2:
        y1 = np.min(y)
        y2 = np.max(y)
    else:
        y1 = 0
        y2 = img.shape[0]
    
    return l, y1, y2, height

def sliding_one(img, nwindows=9, margin=150, minpix = 1, draw_windows=True, offset=0, path=(False, False)):
    """
    offset -> mask offset from x=0
    skretanje lijevo -> True, False
    """
    l, r = path
    fitted = False
    #calculate global histogram
    window_height = np.int(img.shape[0]/nwindows)
    histogram = np.sum(img, axis=0)
    
    #first window starting point
    base = np.argmax(histogram)
    x_current = base
    
    out_img = np.dstack((img, img, img))*255
    #nz
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Create empty lists to receive left and right lane pixel indices
    x_coords = []
    
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_x_low = x_current - margin
        win_x_high = x_current + margin
        
        # Draw the windows on the visualization image
        if draw_windows == True:
            cv2.rectangle(out_img,(win_x_low,win_y_low),(win_x_high,win_y_high),
            (100,255,255), 3)

        x_coords.append(offset+x_current)        
            
        #next window starting position
        if l:   #force left steering
            x_h = x_current
            x_l = 0 if int(x_h-(margin*2)) < 0 else int(x_h-(margin*3))
        elif r:
            x_l = x_current
            x_h = int(img.shape[1]) if int(x_l+(margin*2)) > int(img.shape[1]) else int(x_l+(margin*3))
        else:
            x_l = 0 if int(win_x_low-margin) < 0 else int(win_x_low-margin)
            x_h = int(img.shape[1]) if int(win_x_high+margin) > int(img.shape[1]) else int(win_x_high+margin)
        
        next_hist = np.sum(img[(win_y_low-window_height):(win_y_low), x_l:x_h], axis=0)
        
        if np.sum(next_hist)>2550:  #minimum lane width = 10px
            hig = np.argmax(next_hist)
            x_current = x_l+hig
            fitted = True          

    return out_img, x_coords, fitted

def get_frame(cam, M):
    frame = cam.value.copy()
    frame = cv2.resize(frame, (width, height))
    warp = cv2.warpPerspective(frame, M, (width,height), flags=cv2.INTER_LINEAR)
    
    return warp
    
def get_direction():
    global left_s, right_s
    while True:
        s = get_steering()
        if s is not None:
            if s==-1:
                left_s, right_s = (False, True)
            elif s==1:
                left_s, right_s = (True, False)
            else:
                left_s, right_s = (False, False)
        time.sleep(0.0625)
        
def start_dir():
    x = threading.Thread(target=get_direction)
    x.start()

In [3]:
#gps globals
http = urllib3.PoolManager()
url = 'http://192.168.1.164:8080/get_car_pos'
gpsData =  {"id": 0, "gpsX": -1, "gpsY": -1}
pos_l = (0, 0)
hedge = MarvelmindHedge(tty = "/dev/ttyACM0", adr=None, debug=False) # create MarvelmindHedge thread
hedge.start() # start thread

def get_steering():
    global url, gpsData
    response = None
    pos = hedge.position()
    if (pos[1] !=0 and pos[2] !=0) and pos_l!=pos:
        gpsData = {"id": 0, "gpsX": pos[1], "gpsY" :pos[2]}
        print("GPS:", pos[1], pos[2])
    
    try:
        response = http.request("GET",
                        "http://192.168.1.164:8080/get_car_pos", 
                        headers={"Content-Type": "application/json"},
                        body= json.dumps(gpsData))
    except: 
        #print(f"Server error.")
        return None

    if response.status != 200:
        #print("Invalid server response.") 
        return None

    carInfo = response.data.decode("utf-8")
    carInfo = json.loads(carInfo)
    throttle = float(carInfo["throttle"])
    steering = float(carInfo["steering"])
    #print("Server steering: ", steering)

    return steering

In [4]:
#prefs
show_widgets = True   #show lanes in widgets


#steering
l_steer = 0.0

#car object
car = NvidiaRacecar()

#start camera


#frame size
width  = 398
height = 224
mid = 199   #picture midpoint
mid_offset = 45

#perspective transform matrix
#src=np.float32([[30, 77], [0,102], [389, 102], [340, 62]])
#dst=np.float32([[0,0],[0,224],[398, 224],[398,0]])

src=np.float32([[38, 61], [0,91], [387, 91], [346, 61]])
dst=np.float32([[0,0],[0,224],[398, 224],[398,0]])

M = cv2.getPerspectiveTransform(src, dst)
M_inv = cv2.getPerspectiveTransform(dst, src)


#widgets
if show_widgets:
    w1 = ipywidgets.Image(width=width, height=height)
    w2 = ipywidgets.Image(width=width, height=height)
    w3 = ipywidgets.Image(width=width, height=height)
    w4 = ipywidgets.Image(width=width, height=height)
    
    widgets = ipywidgets.VBox([
        ipywidgets.HBox([w1, w2, w3, w4])
    ])
    display(widgets)

In [5]:
car.throttle = -0.25
y_num = 0
start_dir()
left, right = (False, False)

while True:
    out_img = np.array([0])
    warp = get_frame(camera, M)
    
    fitted = False
    if updated:
        left, right = (left_s, right_s)
        updated = False
    
    #get masks
    masky = yellowMask(warp)
    maskw = whiteMask(warp)    
    
    if (left or right) or not last_fit:
        
        #fallback to yellow lines
        if not last_fit and not (left or right):
            right = True
        
        #get yellow mask
        right_half_y = masky[:, 150:]
        left_half_y = masky[:, :-150]
        
        #if path is left
        if left:
            #check if right line exists
            rf, y1r ,y2r, hr = line_check(right_half_y, n=40)
            
            #get sliding windows
            if rf:
                out_img, xr, fitted = sliding_one(right_half_y[y1r:y2r, :], nwindows=4, margin=25, minpix = 1, draw_windows=show_widgets, offset = 150, path=(left,right))
                if fitted:
                    steer = parse_steering(xr, mid, 167, 154, (False, True, False))
            
        elif right:
            #check if left line exists
            lf, y1l ,y2l, hr = line_check(left_half_y, n=40)
            
            #get sliding windows
            if lf:
                out_img, xl, fitted = sliding_one(left_half_y[y1l:y2l, :], nwindows=4, margin=25, minpix = 1, draw_windows=show_widgets, path=(left, right))
                if fitted:
                    steer = parse_steering(xl, mid, 167, 154, (True, False, False))
        
        #if yellow line was not found, check for white in middle of frame
        if not fitted:
            y_num+=1
            
        if fitted:
            y_num = 0
            
    if (not fitted and y_num >2) or (not left and not right):
        #get white mask and slice
        
        left_half_w = maskw[:, :(mid-mid_offset)]
        right_half_w = maskw[:, (mid+mid_offset):]
        mid_part_w = maskw[:, int((mid/2)):int(mid+(mid/2))]
        
        #check for white lines
        l, y1l ,y2l, hl = line_check(left_half_w, n=60)
        r, y1r ,y2r, hr = line_check(right_half_w, n=60)
        
        print("LR white: ", l, r)
        #get sliding windows
        if l and (hl>=hr+10 or not r):
            out_img, xl, fitted = sliding_one(left_half_w[y1l:y2l, :], nwindows=4, margin=25, minpix = 1, draw_windows=show_widgets)
            if fitted:
                steer = parse_steering(xl, mid, 167, 154, (True, False, False))
            
        elif r:
            out_img, xr, fitted = sliding_one(right_half_w[y1r:y2r, :], nwindows=4, margin=25, minpix = 1, draw_windows=show_widgets, offset = mid+mid_offset)
            if fitted:
                steer = parse_steering(xr, mid, 167, 154, (False, True, False))
                
        else:
            m, y1, y2, hm = line_check(mid_part_w, n=50)
            #m = False
            if m:
                out_img, xm, fitted = sliding_one(mid_part_w[y1:y2, :], nwindows=4, margin=25, minpix = 1, draw_windows=show_widgets, offset =  int((mid/2)))
                steer = parse_steering(xm, mid, 167, 154, (False, False, True))
                print(xm)
            else:
                steer = l_steer
            
    elif not fitted:
        print("No lines", y_num)
        steer = l_steer
    
    #print vars
    print("S: ", steer, l_steer)
    last_fit = fitted
    
    #steer car
    steer = (l_steer+steer)/2
    l_steer = steer
    car.steering = steer
    
    #update widgets
    if show_widgets:
        w1.value = bgr8_to_jpeg(warp)
        #w2.value = bgr8_to_jpeg()
        if left or right:
            w2.value = bgr8_to_jpeg(masky)
        else:
            w2.value = bgr8_to_jpeg(maskw)
        if fitted:
            w3.value = bgr8_to_jpeg(out_img)
    #clear_output(True)

GPS: -6.026 -1.58
No lines 1
S:  0.0 0.0
No lines 2
S:  0.0 0.0
LR white:  True True
S:  0.15577889447236182 0.0
LR white:  True True
S:  0.15577889447236182 0.07788944723618091
LR white:  True True
S:  0.15577889447236182 0.11683417085427136
LR white:  True True
S:  0.15577889447236182 0.1363065326633166
LR white:  True True
S:  0.15577889447236182 0.1460427135678392
LR white:  True True
S:  0.15577889447236182 0.15091080402010051
LR white:  True True
S:  0.15577889447236182 0.15334484924623115
LR white:  True True
S:  0.15577889447236182 0.15456187185929648
GPS: -6.028 -1.579
LR white:  True True
S:  0.15577889447236182 0.15517038316582915
LR white:  True True
S:  0.15577889447236182 0.15547463881909548
LR white:  True True
S:  0.15577889447236182 0.15562676664572866
LR white:  True True
S:  0.15577889447236182 0.15570283055904524
LR white:  True True
S:  0.15577889447236182 0.15574086251570352
LR white:  True True
S:  0.15577889447236182 0.15575987849403267
LR white:  True True
S:  

KeyboardInterrupt: 

In [ ]:
car.throttle = 0

GPS: -5.94 -1.311
GPS: -5.95 -1.313
GPS: -5.967 -1.313
GPS: -5.961 -1.31
GPS: -5.951 -1.307
GPS: -5.942 -1.3
GPS: -5.947 -1.301
GPS: -5.946 -1.309
GPS: -5.955 -1.314
GPS: -5.953 -1.311
GPS: -5.943 -1.307
GPS: -5.944 -1.306
GPS: -5.968 -1.313
GPS: -5.972 -1.312
GPS: -5.984 -1.314
GPS: -5.987 -1.315
GPS: -5.974 -1.314
GPS: -5.964 -1.311
GPS: -5.958 -1.309
GPS: -5.953 -1.31
GPS: -5.949 -1.31
GPS: -5.954 -1.31
GPS: -5.937 -1.304
GPS: -5.927 -1.3
GPS: -5.931 -1.305
GPS: -5.936 -1.307
GPS: -5.93 -1.303
GPS: -5.933 -1.305
GPS: -5.932 -1.307
GPS: -5.951 -1.31
GPS: -5.938 -1.312
GPS: -5.938 -1.312
GPS: -5.92 -1.308
GPS: -5.921 -1.305
GPS: -5.922 -1.303
GPS: -5.933 -1.305
GPS: -5.942 -1.311
GPS: -5.927 -1.299
GPS: -5.921 -1.292
GPS: -5.925 -1.305
GPS: -5.941 -1.306
GPS: -5.964 -1.315
GPS: -5.948 -1.316
GPS: -5.945 -1.314
GPS: -5.945 -1.314
GPS: -5.938 -1.312
GPS: -5.934 -1.311
GPS: -5.931 -1.308
GPS: -5.925 -1.302
GPS: -5.923 -1.299
GPS: -5.935 -1.308
GPS: -5.942 -1.31
GPS: -5.941 -1.31
GPS: -5.